In [2]:
import os
import sys
import torch
import torch.utils.data as data
import numpy as np
from PIL import Image
import glob
import random
import cv2

from tqdm import tqdm

import torch.nn as nn
import torch.nn.functional as F
import math


from torchvision.models.vgg import vgg16


import torchvision
import torch.backends.cudnn as cudnn
import torch.optim
import time
from torchvision import transforms

#  DataLoader

In [2]:
random.seed(1103)


def populate_train_list(lowlight_images_path):
    image_list_lowlight = glob.glob(lowlight_images_path + "/*.jpg")
    
    train_list = image_list_lowlight
    random.shuffle(train_list)

    return train_list



class lowlight_loader(data.Dataset):

    def __init__(self, lowlight_images_path):

        self.train_list = populate_train_list(lowlight_images_path) 
        self.size = 256

        self.data_list = self.train_list
        print("Total training examples:", len(self.train_list))


    def __getitem__(self, index):

        data_lowlight_path = self.data_list[index]

        data_lowlight = Image.open(data_lowlight_path)

        data_lowlight = data_lowlight.resize((self.size,self.size), Image.Resampling.LANCZOS)

        data_lowlight = (np.asarray(data_lowlight)/255.0) 
        data_lowlight = torch.from_numpy(data_lowlight).float()

        return data_lowlight.permute(2,0,1)
    
    def __len__(self):
        return len(self.data_list)

# **Model**

In [3]:
class enhance_net_nopool(nn.Module):

    def __init__(self):
        super(enhance_net_nopool, self).__init__()

        self.relu = nn.ReLU(inplace=True)

        number_f = 32
        self.e_conv1 = nn.Conv2d(3,number_f,3,1,1,bias=True) 
        self.e_conv2 = nn.Conv2d(number_f,number_f,3,1,1,bias=True) 
        self.e_conv3 = nn.Conv2d(number_f,number_f,3,1,1,bias=True) 
        self.e_conv4 = nn.Conv2d(number_f,number_f,3,1,1,bias=True) 
        self.e_conv5 = nn.Conv2d(number_f*2,number_f,3,1,1,bias=True) 
        self.e_conv6 = nn.Conv2d(number_f*2,number_f,3,1,1,bias=True) 
        self.e_conv7 = nn.Conv2d(number_f*2,24,3,1,1,bias=True) 

        self.maxpool = nn.MaxPool2d(2, stride=2, return_indices=False, ceil_mode=False)
        self.upsample = nn.UpsamplingBilinear2d(scale_factor=2)



    def forward(self, x):

        x1 = self.relu(self.e_conv1(x))
        # p1 = self.maxpool(x1)
        x2 = self.relu(self.e_conv2(x1))
        # p2 = self.maxpool(x2)
        x3 = self.relu(self.e_conv3(x2))
        # p3 = self.maxpool(x3)
        x4 = self.relu(self.e_conv4(x3))

        x5 = self.relu(self.e_conv5(torch.cat([x3,x4],1)))
        # x5 = self.upsample(x5)
        x6 = self.relu(self.e_conv6(torch.cat([x2,x5],1)))

        x_r = F.tanh(self.e_conv7(torch.cat([x1,x6],1)))
        r1,r2,r3,r4,r5,r6,r7,r8 = torch.split(x_r, 3, dim=1)


        x = x + r1*(torch.pow(x,2)-x)
        x = x + r2*(torch.pow(x,2)-x)
        x = x + r3*(torch.pow(x,2)-x)
        enhance_image_1 = x + r4*(torch.pow(x,2)-x)		
        x = enhance_image_1 + r5*(torch.pow(enhance_image_1,2)-enhance_image_1)		
        x = x + r6*(torch.pow(x,2)-x)	
        x = x + r7*(torch.pow(x,2)-x)
        enhance_image = x + r8*(torch.pow(x,2)-x)
        r = torch.cat([r1,r2,r3,r4,r5,r6,r7,r8],1)
        return enhance_image_1,enhance_image,r
        #return enhance_image
        #return r
        



# **MYLOSS**

In [4]:
class L_color(nn.Module):

    def __init__(self):
        super(L_color, self).__init__()

    def forward(self, x ):

        b,c,h,w = x.shape

        mean_rgb = torch.mean(x,[2,3],keepdim=True)
        mr,mg, mb = torch.split(mean_rgb, 1, dim=1)
        Drg = torch.pow(mr-mg,2)
        Drb = torch.pow(mr-mb,2)
        Dgb = torch.pow(mb-mg,2)
        k = torch.pow(torch.pow(Drg,2) + torch.pow(Drb,2) + torch.pow(Dgb,2),0.5)


        return k

class L_spa(nn.Module):

    def __init__(self):
        super(L_spa, self).__init__()
        # print(1)kernel = torch.FloatTensor(kernel).unsqueeze(0).unsqueeze(0)
        kernel_left = torch.FloatTensor( [[0,0,0],[-1,1,0],[0,0,0]]).to(device).unsqueeze(0).unsqueeze(0)
        kernel_right = torch.FloatTensor( [[0,0,0],[0,1,-1],[0,0,0]]).to(device).unsqueeze(0).unsqueeze(0)
        kernel_up = torch.FloatTensor( [[0,-1,0],[0,1, 0 ],[0,0,0]]).to(device).unsqueeze(0).unsqueeze(0)
        kernel_down = torch.FloatTensor( [[0,0,0],[0,1, 0],[0,-1,0]]).to(device).unsqueeze(0).unsqueeze(0)
        self.weight_left = nn.Parameter(data=kernel_left, requires_grad=False)
        self.weight_right = nn.Parameter(data=kernel_right, requires_grad=False)
        self.weight_up = nn.Parameter(data=kernel_up, requires_grad=False)
        self.weight_down = nn.Parameter(data=kernel_down, requires_grad=False)
        self.pool = nn.AvgPool2d(4)
    def forward(self, org , enhance ):
        b,c,h,w = org.shape

        org_mean = torch.mean(org,1,keepdim=True)
        enhance_mean = torch.mean(enhance,1,keepdim=True)

        org_pool =  self.pool(org_mean)			
        enhance_pool = self.pool(enhance_mean)	

        weight_diff =torch.max(torch.FloatTensor([1]).to(device) + 10000*torch.min(org_pool - torch.FloatTensor([0.3]).to(device),torch.FloatTensor([0]).to(device)),torch.FloatTensor([0.5]).to(device))
        E_1 = torch.mul(torch.sign(enhance_pool - torch.FloatTensor([0.5]).to(device)) ,enhance_pool-org_pool)


        D_org_letf = F.conv2d(org_pool , self.weight_left, padding=1)
        D_org_right = F.conv2d(org_pool , self.weight_right, padding=1)
        D_org_up = F.conv2d(org_pool , self.weight_up, padding=1)
        D_org_down = F.conv2d(org_pool , self.weight_down, padding=1)

        D_enhance_letf = F.conv2d(enhance_pool , self.weight_left, padding=1)
        D_enhance_right = F.conv2d(enhance_pool , self.weight_right, padding=1)
        D_enhance_up = F.conv2d(enhance_pool , self.weight_up, padding=1)
        D_enhance_down = F.conv2d(enhance_pool , self.weight_down, padding=1)

        D_left = torch.pow(D_org_letf - D_enhance_letf,2)
        D_right = torch.pow(D_org_right - D_enhance_right,2)
        D_up = torch.pow(D_org_up - D_enhance_up,2)
        D_down = torch.pow(D_org_down - D_enhance_down,2)
        E = (D_left + D_right + D_up +D_down)
        # E = 25*(D_left + D_right + D_up +D_down)

        return E
class L_exp(nn.Module):

    def __init__(self,patch_size,mean_val):
        super(L_exp, self).__init__()
        # print(1)
        self.pool = nn.AvgPool2d(patch_size)
        self.mean_val = mean_val
    def forward(self, x ):

        b,c,h,w = x.shape
        x = torch.mean(x,1,keepdim=True)
        mean = self.pool(x)

        d = torch.mean(torch.pow(mean- torch.FloatTensor([self.mean_val] ).to(device),2))
        return d
        
class L_TV(nn.Module):
    def __init__(self,TVLoss_weight=1):
        super(L_TV,self).__init__()
        self.TVLoss_weight = TVLoss_weight

    def forward(self,x):
        batch_size = x.size()[0]
        h_x = x.size()[2]
        w_x = x.size()[3]
        count_h =  (x.size()[2]-1) * x.size()[3]
        count_w = x.size()[2] * (x.size()[3] - 1)
        h_tv = torch.pow((x[:,:,1:,:]-x[:,:,:h_x-1,:]),2).sum()
        w_tv = torch.pow((x[:,:,:,1:]-x[:,:,:,:w_x-1]),2).sum()
        return self.TVLoss_weight*2*(h_tv/count_h+w_tv/count_w)/batch_size

# Camera

In [2]:
gst_str = ('nvarguscamerasrc ! '
            'video/x-raw(memory:NVMM), '
            'width=(int)1280, height=(int)720, '
            'format=(string)NV12, framerate=(fraction)30/1 ! '
            'nvvidconv flip-method=0 ! '
            'video/x-raw, width=(int){}, height=(int){}, '
            'format=(string)BGRx ! '
            'videoconvert ! appsink').format(1280, 720)  #width, height
cap = cv2.VideoCapture(gst_str)
if not cap.isOpened():
   print('Failed to open camera!')
flag = cap.isOpened()
index = 1
while (flag):
    ret, frame = cap.read()
    cv2.imshow("Capture_Paizhao", frame)
    k = cv2.waitKey(1) & 0xFF
    if k == ord('s'):  # 按下s键，进入下面的保存图片操作
        cv2.imwrite("./test_data/NEW/NEW"+str(index)+".jpg", frame)
        print("save " + "NEW"+str(index) + ".jpg successfuly!")
        print("-------------------------")
        index += 1
    elif k  & 0xFF == 27:  # 按下esc键，程序退出
        break
cap.release() # 释放摄像头
cv2.destroyAllWindows()# 释放并销毁窗口

# vedio打开摄像头录制

In [3]:
gst_str = ('nvarguscamerasrc ! '
            'video/x-raw(memory:NVMM), '
            'width=(int)480, height=(int)640, '
            'format=(string)NV12, framerate=(fraction)20/1 ! '
            'nvvidconv flip-method=0 ! '
            'video/x-raw, width=(int){}, height=(int){}, '
            'format=(string)BGRx ! '
            'videoconvert ! appsink').format(480, 640)  #width, height

cap = cv2.VideoCapture(gst_str)
if not cap.isOpened():
   print('Failed to open camera!')
flag = cap.isOpened()
index = 1
while (flag):
    ret, frame = cap.read()
    cv2.imshow("Capture", frame)
    cv2.imwrite('/home/tx2/Zero-DCE/test_data/New_vedio/NewVedio%d.jpg'%index, frame)
    index += 1
    if cv2.waitKey(1) & 0xFF == 27 :
        break 
cap.release() # 释放摄像头
cv2.destroyAllWindows()# 释放并销毁窗口

# vedio选择TX2中路径

In [5]:
# import matplotlib.pyplot as plt
# plt.xticks([]), plt.yticks([]) # 隐藏x和y轴
index=1
#视频读取
#cv2.VideoCapture可以捕获摄像头，用数字来控制不同的设备，例如0,1。
#如果是视频文件，直接指定好路径即可。

#VideoCapture()中参数是0，表示打开笔记本的内置摄像头。
#video = cv.VideoCapture(0)

#表示参数是视频文件路径则打开视频
video = cv2.VideoCapture('/home/tx2/Zero-DCE/test_vedio/test.mp4')

#检查是否正确打开isOpened()
#循环读取每一帧
while video.isOpened():
    #video.read() : 一次读取视频中的每一帧，会返回两个值；
    #res : 为bool类型表示这一帧是否真确读取，正确读取为True，如果文件读取到结尾，它的返回值就为False;
    #frame : 表示这一帧的像素点数组
    ret, frame = video.read()
    if frame is None: 
        break
    if ret == True:
        cv2.imwrite('/home/tx2/Zero-DCE/test_data/Vedio_pic/Vedio%d.jpg'%index, frame)
        index=index+1
#         img_new2 = frame[:, :, ::-1]
#         plt.imshow(img_new2)
        cv2.imshow("result", frame)
    #100 ： 表示一帧等待一百毫秒在进入下一帧， 0xFF : 表示键入键 27 = esc
    if cv2.waitKey(50) & 0xFF == 27 :
        break 
#video.release()释放视频
video.release()
cv2.destroyAllWindows()

# **TEST**

In [8]:
import torch
import torch.nn as nn
import torchvision
import torch.backends.cudnn as cudnn
import torch.optim
import os
import sys
import argparse
import time
import numpy as np
from torchvision import transforms
from PIL import Image
import glob
import time


 
def lowlight(image_path):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    data_lowlight = Image.open(image_path)



    data_lowlight = (np.asarray(data_lowlight)/255.0)


    data_lowlight = torch.from_numpy(data_lowlight).float()
    data_lowlight = data_lowlight.permute(2,0,1)
    data_lowlight = data_lowlight.to(device).unsqueeze(0)

    DCE_net = enhance_net_nopool().to(device)
    DCE_net.load_state_dict(torch.load('./snapshots/Epoch200.pth'))
    start = time.time()
    _,enhanced_image,_ = DCE_net(data_lowlight)

    end_time = (time.time() - start)
    print(end_time)
    result_path = "./working/"
    print(result_path+image_path.split("/")[-1])
    torchvision.utils.save_image(enhanced_image, result_path+image_path.split("/")[-1])

    
with torch.no_grad():
    filePath = './test_data/'

    file_list = os.listdir(filePath)

    for file_name in file_list:
        test_list = glob.glob(filePath+file_name+"/*") 
        for image in test_list:
            # image = image
            print(image)
            lowlight(image)

print("finish!")



./test_data/New_vedio/NewVedio173.jpg


/home/tx2/.local/lib/python3.6/site-packages/torch/nn/functional.py:1794: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


2.7223942279815674
./working/NewVedio173.jpg
./test_data/New_vedio/NewVedio244.jpg
0.005994558334350586
./working/NewVedio244.jpg
./test_data/New_vedio/NewVedio163.jpg
0.006193876266479492
./working/NewVedio163.jpg
./test_data/New_vedio/NewVedio245.jpg
0.006075143814086914
./working/NewVedio245.jpg
./test_data/New_vedio/NewVedio64.jpg
0.006221294403076172
./working/NewVedio64.jpg
./test_data/New_vedio/NewVedio116.jpg
0.0071642398834228516
./working/NewVedio116.jpg
./test_data/New_vedio/NewVedio193.jpg
0.006154298782348633
./working/NewVedio193.jpg
./test_data/New_vedio/NewVedio100.jpg
0.006048679351806641
./working/NewVedio100.jpg
./test_data/New_vedio/NewVedio180.jpg
0.006140708923339844
./working/NewVedio180.jpg
./test_data/New_vedio/NewVedio118.jpg
0.0062177181243896484
./working/NewVedio118.jpg
./test_data/New_vedio/NewVedio136.jpg
0.006102800369262695
./working/NewVedio136.jpg
./test_data/New_vedio/NewVedio105.jpg
0.006130218505859375
./working/NewVedio105.jpg
./test_data/New_vedi

./test_data/New_vedio/NewVedio33.jpg
0.006523609161376953
./working/NewVedio33.jpg
./test_data/New_vedio/NewVedio70.jpg
0.011472225189208984
./working/NewVedio70.jpg
./test_data/New_vedio/NewVedio216.jpg
0.012633323669433594
./working/NewVedio216.jpg
./test_data/New_vedio/NewVedio157.jpg
0.007355690002441406
./working/NewVedio157.jpg
./test_data/New_vedio/NewVedio218.jpg
0.01129293441772461
./working/NewVedio218.jpg
./test_data/New_vedio/NewVedio219.jpg
0.00631403923034668
./working/NewVedio219.jpg
./test_data/New_vedio/NewVedio98.jpg
0.00760650634765625
./working/NewVedio98.jpg
./test_data/New_vedio/NewVedio126.jpg
0.0060863494873046875
./working/NewVedio126.jpg
./test_data/New_vedio/NewVedio37.jpg
0.007130622863769531
./working/NewVedio37.jpg
./test_data/New_vedio/NewVedio95.jpg
0.006699323654174805
./working/NewVedio95.jpg
./test_data/New_vedio/NewVedio69.jpg
0.00846719741821289
./working/NewVedio69.jpg
./test_data/New_vedio/NewVedio211.jpg
0.007510185241699219
./working/NewVedio211

./test_data/New_vedio/NewVedio205.jpg
0.006338357925415039
./working/NewVedio205.jpg
./test_data/New_vedio/NewVedio101.jpg
0.006169795989990234
./working/NewVedio101.jpg
./test_data/New_vedio/NewVedio248.jpg
0.00606083869934082
./working/NewVedio248.jpg
./test_data/New_vedio/NewVedio86.jpg
0.0060079097747802734
./working/NewVedio86.jpg
./test_data/New_vedio/NewVedio45.jpg
0.0060117244720458984
./working/NewVedio45.jpg
./test_data/New_vedio/NewVedio85.jpg
0.006251811981201172
./working/NewVedio85.jpg
./test_data/New_vedio/NewVedio110.jpg
0.006284475326538086
./working/NewVedio110.jpg
./test_data/New_vedio/NewVedio46.jpg
0.006222724914550781
./working/NewVedio46.jpg
./test_data/New_vedio/NewVedio40.jpg
0.006155967712402344
./working/NewVedio40.jpg
./test_data/New_vedio/NewVedio82.jpg
0.006742715835571289
./working/NewVedio82.jpg
./test_data/New_vedio/NewVedio137.jpg
0.006380558013916016
./working/NewVedio137.jpg
./test_data/New_vedio/NewVedio155.jpg
0.012650012969970703
./working/NewVedi

./test_data/Vedio_pic/Vedio147.jpg
0.006163120269775391
./working/Vedio147.jpg
./test_data/Vedio_pic/Vedio20.jpg
0.005925416946411133
./working/Vedio20.jpg
./test_data/Vedio_pic/Vedio43.jpg
0.006227016448974609
./working/Vedio43.jpg
./test_data/Vedio_pic/Vedio73.jpg
0.00850820541381836
./working/Vedio73.jpg
./test_data/Vedio_pic/Vedio77.jpg
0.0062253475189208984
./working/Vedio77.jpg
./test_data/Vedio_pic/Vedio129.jpg
0.005964756011962891
./working/Vedio129.jpg
./test_data/Vedio_pic/Vedio84.jpg
0.0060214996337890625
./working/Vedio84.jpg
./test_data/Vedio_pic/Vedio91.jpg
0.00656890869140625
./working/Vedio91.jpg
./test_data/Vedio_pic/Vedio31.jpg
0.005978107452392578
./working/Vedio31.jpg
./test_data/Vedio_pic/Vedio71.jpg
0.005939006805419922
./working/Vedio71.jpg
./test_data/Vedio_pic/Vedio95.jpg
0.00594639778137207
./working/Vedio95.jpg
./test_data/Vedio_pic/Vedio45.jpg
0.006166934967041016
./working/Vedio45.jpg
./test_data/Vedio_pic/Vedio136.jpg
0.007906198501586914
./working/Vedio13

./test_data/Vedio_pic/Vedio70.jpg
0.006665945053100586
./working/Vedio70.jpg
./test_data/Vedio_pic/Vedio88.jpg
0.006119966506958008
./working/Vedio88.jpg
./test_data/Vedio_pic/Vedio114.jpg
0.00608515739440918
./working/Vedio114.jpg
./test_data/Vedio_pic/Vedio48.jpg
0.005862236022949219
./working/Vedio48.jpg
./test_data/Vedio_pic/Vedio55.jpg
0.009128808975219727
./working/Vedio55.jpg
./test_data/Vedio_pic/Vedio5.jpg
0.006156444549560547
./working/Vedio5.jpg
./test_data/Vedio_pic/Vedio9.jpg
0.008073568344116211
./working/Vedio9.jpg
./test_data/Vedio_pic/Vedio123.jpg
0.0062427520751953125
./working/Vedio123.jpg
./test_data/Vedio_pic/Vedio47.jpg
0.0062563419342041016
./working/Vedio47.jpg
./test_data/DICM/35.jpg
0.07317590713500977
./working/35.jpg
./test_data/DICM/13.jpg
0.00769495964050293
./working/13.jpg
./test_data/DICM/02.jpg
0.005685091018676758
./working/02.jpg
./test_data/DICM/28.jpg
0.0059697628021240234
./working/28.jpg
./test_data/DICM/39.jpg
0.0077893733978271484
./working/39.

# 选择路径视频的合成

In [7]:
files = os.listdir("/home/tx2/Zero-DCE/test_data/Vedio_pic/")
# 要被合成的多张图片所在文件夹
# 路径分隔符最好使用“/”,而不是“\”,“\”本身有转义的意思；或者“\\”也可以。
# 因为是文件夹，所以最后还要有一个“/”
# VideoWriter是cv2库提供的视频保存方法，将合成的视频保存到该路径中
# 'MJPG'意思是支持jpg格式图片
# fps = 5代表视频的帧频为5，如果图片不多，帧频最好设置的小一点
# (1280,720)是生成的视频像素1280*720，一般要与所使用的图片像素大小一致，否则生成的视频无法播放
# 定义保存视频目录名称和压缩格式，像素为1280*720
video = cv2.VideoWriter('/home/tx2/Zero-DCE/test_vedio/enhance_test.mp4',cv2.VideoWriter_fourcc(*'mp4v'),30,(1280,720))

for i in range(1,len(files)):
    #读取图片
    img = cv2.imread('/home/tx2/Zero-DCE/working/Vedio%d.jpg'%i)     
   	# resize方法是cv2库提供的更改像素大小的方法
    # 将图片转换为1280*720像素大小
    #img = cv2.resize(img,(1280,720))
    # 写入视频
    video.write(img)

# 释放资源
video.release()
print("finish!")

finish!


# 打开摄像头录制视频的合成

In [21]:
files = os.listdir("/home/tx2/Zero-DCE/test_data/New_vedio/")
# 要被合成的多张图片所在文件夹
# 路径分隔符最好使用“/”,而不是“\”,“\”本身有转义的意思；或者“\\”也可以。
# 因为是文件夹，所以最后还要有一个“/”
# VideoWriter是cv2库提供的视频保存方法，将合成的视频保存到该路径中
# 'MJPG'意思是支持jpg格式图片
# fps = 5代表视频的帧频为5，如果图片不多，帧频最好设置的小一点
# (1280,720)是生成的视频像素1280*720，一般要与所使用的图片像素大小一致，否则生成的视频无法播放
# 定义保存视频目录名称和压缩格式，像素为1280*720
video = cv2.VideoWriter('/home/tx2/Zero-DCE/test_vedio/new_test.mp4',cv2.VideoWriter_fourcc(*'mp4v'),30,(1280,720))

for i in range(1,len(files)):
    #读取图片
    img = cv2.imread('/home/tx2/Zero-DCE/test_data/New_vedio/NewVedio%d.jpg'%i)     
   	# resize方法是cv2库提供的更改像素大小的方法
    # 将图片转换为1280*720像素大小
    #img = cv2.resize(img,(1280,720))
    # 写入视频
    video.write(img)

# 释放资源
video.release()

video = cv2.VideoWriter('/home/tx2/Zero-DCE/test_vedio/enhance_new_test.mp4',cv2.VideoWriter_fourcc(*'mp4v'),30,(1280,720))

for i in range(1,len(files)):
    #读取图片
    img = cv2.imread('/home/tx2/Zero-DCE/working/NewVedio%d.jpg'%i)     
   	# resize方法是cv2库提供的更改像素大小的方法
    # 将图片转换为1280*720像素大小
    #img = cv2.resize(img,(1280,720))
    # 写入视频
    video.write(img)

# 释放资源
video.release()
print("finish!")


finish!


# 实时视频增强--Torch版

In [5]:
import torch
import torch.nn as nn
import torchvision
import torch.backends.cudnn as cudnn
import torch.optim
import os
import sys
import argparse
import time
import numpy as np
from torchvision import transforms
from PIL import Image
import glob
import time


 
def lowlight(image_path):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    data_lowlight = Image.open(image_path)



    data_lowlight = (np.asarray(data_lowlight)/255.0)


    data_lowlight = torch.from_numpy(data_lowlight).float()
    data_lowlight = data_lowlight.permute(2,0,1)
    data_lowlight = data_lowlight.to(device).unsqueeze(0)

    DCE_net = enhance_net_nopool().to(device)
    DCE_net.load_state_dict(torch.load('./snapshots/Epoch200.pth'))
    start = time.time()
    _,enhanced_image,_ = DCE_net(data_lowlight)

    end_time = (time.time() - start)
    print(end_time)
    result_path = "./working/"
    print(result_path+image_path.split("/")[-1])
    torchvision.utils.save_image(enhanced_image, result_path+image_path.split("/")[-1])
    
gst_str = ('nvarguscamerasrc ! '
            'video/x-raw(memory:NVMM), '
            'width=(int)1280, height=(int)720, '
            'format=(string)NV12, framerate=(fraction)30/1 ! '
            'nvvidconv flip-method=0 ! '
            'video/x-raw, width=(int){}, height=(int){}, '
            'format=(string)BGRx ! '
            'videoconvert ! appsink').format(1280, 720)  #width, height

cap = cv2.VideoCapture(gst_str)
if not cap.isOpened():
   print('Failed to open camera!')
flag = cap.isOpened()
i = 1
while (flag):
    ret, frame = cap.read()
    cv2.imshow("Capture", frame)
    cv2.imwrite("./test_data/actual_vedio/actual_vedio%d.jpg"%i, frame)
    lowlight("./test_data/actual_vedio/actual_vedio%d.jpg"%i)
    i=i+1
    retval = cap.get(5)
    if cv2.waitKey(33) & 0xFF == 27 :
        break 
cap.release() # 释放摄像头
cv2.destroyAllWindows()# 释放并销毁窗口  




files = os.listdir("/home/tx2/Zero-DCE/test_data/actual_vedio/")
# 要被合成的多张图片所在文件夹
# 路径分隔符最好使用“/”,而不是“\”,“\”本身有转义的意思；或者“\\”也可以。
# 因为是文件夹，所以最后还要有一个“/”
# VideoWriter是cv2库提供的视频保存方法，将合成的视频保存到该路径中
# 'MJPG'意思是支持jpg格式图片
# fps = 5代表视频的帧频为5，如果图片不多，帧频最好设置的小一点
# (1280,720)是生成的视频像素1280*720，一般要与所使用的图片像素大小一致，否则生成的视频无法播放
# 定义保存视频目录名称和压缩格式，像素为1280*720
video = cv2.VideoWriter('/home/tx2/Zero-DCE/test_vedio/actual_test.mp4',cv2.VideoWriter_fourcc(*'mp4v'),30,(1280,720))

for i in range(1,len(files)):
    #读取图片
    img = cv2.imread('./working/actual_vedio%d.jpg'%i)     
   	# resize方法是cv2库提供的更改像素大小的方法
    # 将图片转换为1280*720像素大小
    #img = cv2.resize(img,(1280,720))
    # 写入视频
    video.write(img)

# 释放资源
video.release()
print("finish!")

/home/tx2/.local/lib/python3.6/site-packages/torch/nn/functional.py:1794: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


5.042782783508301
./working/actual_vedio1.jpg
0.009857177734375
./working/actual_vedio2.jpg
0.00993490219116211
./working/actual_vedio3.jpg
0.009548187255859375
./working/actual_vedio4.jpg
0.0075778961181640625
./working/actual_vedio5.jpg
0.008338689804077148
./working/actual_vedio6.jpg
0.007040977478027344
./working/actual_vedio7.jpg
0.00673222541809082
./working/actual_vedio8.jpg
0.006707191467285156
./working/actual_vedio9.jpg
0.006955385208129883
./working/actual_vedio10.jpg
0.0070912837982177734
./working/actual_vedio11.jpg
0.008150100708007812
./working/actual_vedio12.jpg
0.010020732879638672
./working/actual_vedio13.jpg
0.009380102157592773
./working/actual_vedio14.jpg
0.013527631759643555
./working/actual_vedio15.jpg
0.01181650161743164
./working/actual_vedio16.jpg
0.010756492614746094
./working/actual_vedio17.jpg
0.010182857513427734
./working/actual_vedio18.jpg
0.008504390716552734
./working/actual_vedio19.jpg
0.012147665023803711
./working/actual_vedio20.jpg
0.00993132591247

# 实时视频增强--TensorRT版

In [5]:
import torch
import torchvision
import numpy as np
import tensorrt as trt
import pycuda.driver as cuda
import pycuda.autoinit
import time
from PIL import Image
import cv2

gst_str = ('nvarguscamerasrc ! '
            'video/x-raw(memory:NVMM), '
            'width=(int)480, height=(int)640, '
            'format=(string)NV12, framerate=(fraction)30/1 ! '
            'nvvidconv flip-method=0 ! '
            'video/x-raw, width=(int){}, height=(int){}, '
            'format=(string)BGRx ! '
            'videoconvert ! appsink').format(480, 640)  #width, height

cap = cv2.VideoCapture(gst_str)
if not cap.isOpened():
   print('Failed to open camera!')
flag = cap.isOpened()
i = 1

def predict(path): # result gets copied into output
    #预处理
    batch = Image.open(path)
    batch = (np.asarray(batch)/255.0)
    batch = np.transpose(batch,(2,0,1))
    batch=np.ascontiguousarray(batch,dtype=np.float32)
    
    # transfer input data to device
    cuda.memcpy_htod_async(d_input, batch, stream)
    # execute model
    context.execute_async_v2(bindings, stream.handle, None)  # 此处采用异步推理。如果想要同步推理，需将execute_async_v2替换成execute_v2
    # transfer predictions back
    cuda.memcpy_dtoh_async(output, d_output, stream)
    # syncronize threads
    stream.synchronize()

    return output



# 1. 确定batch size大小，与导出的trt模型保持一致
BATCH_SIZE = 1        

# 2. 选择是否采用FP16精度，与导出的trt模型保持一致
# USE_FP16 = True                                         
# target_dtype = np.float16 if USE_FP16 else np.float32   
f = open("new_DCE_net_onnx_model.trt", "rb")    
   # 创建一个Runtime(传入记录器Logger)
runtime = trt.Runtime(trt.Logger())
engine = runtime.deserialize_cuda_engine(f.read())      # 从文件中加载trt引擎
context = engine.create_execution_context()             # 创建context


# 4. 分配input和output内存
input_batch = np.random.randn(BATCH_SIZE, 720, 1280, 3).astype(np.float32)
output = np.empty([BATCH_SIZE, 3,720,1280], dtype = np.float32)

d_input = cuda.mem_alloc(1 * input_batch.nbytes)
d_output = cuda.mem_alloc(1 * output.nbytes)

bindings = [int(d_input), int(d_output)]

stream = cuda.Stream()



while (flag):
    ret, frame = cap.read()
    cv2.imshow("Capture", frame)
    cv2.imwrite("./test_data/actual_vedio_tensorrt/actual_vedio_tensorrt%d.jpg"%i, frame)
    image=predict("./test_data/actual_vedio_tensorrt/actual_vedio_tensorrt%d.jpg"%i)
    image=torch.from_numpy(image)
    torchvision.utils.save_image(image, './working/actual_vedio_tensorrt%d.jpg'%i)
    i=i+1
    #retval = cap.get(5)
    if cv2.waitKey(33) & 0xFF == 27 :
        break 
cap.release() # 释放摄像头
cv2.destroyAllWindows()# 释放并销毁窗口  




# files = os.listdir("/home/tx2/Zero-DCE/test_data/actual_vedio_tensorrt/")
# # 要被合成的多张图片所在文件夹
# # 路径分隔符最好使用“/”,而不是“\”,“\”本身有转义的意思；或者“\\”也可以。
# # 因为是文件夹，所以最后还要有一个“/”
# # VideoWriter是cv2库提供的视频保存方法，将合成的视频保存到该路径中
# # 'MJPG'意思是支持jpg格式图片
# # fps = 5代表视频的帧频为5，如果图片不多，帧频最好设置的小一点
# # (1280,720)是生成的视频像素1280*720，一般要与所使用的图片像素大小一致，否则生成的视频无法播放
# # 定义保存视频目录名称和压缩格式，像素为1280*720
# video = cv2.VideoWriter('/home/tx2/Zero-DCE/test_vedio/actual_test_tensorrt.mp4',cv2.VideoWriter_fourcc(*'mp4v'),30,(1280,720))

# for i in range(1,len(files)):
#     #读取图片
#     img = cv2.imread('./working/actual_vedio_tensorrt%d.jpg'%i)     
#    	# resize方法是cv2库提供的更改像素大小的方法
#     # 将图片转换为1280*720像素大小
#     #img = cv2.resize(img,(1280,720))
#     # 写入视频
#     video.write(img)

# # 释放资源
# video.release()
print("finish!")

finish!


# 训练

In [7]:
# def weights_init(m):
#     classname = m.__class__.__name__
#     if classname.find('Conv') != -1:
#         m.weight.data.normal_(0.0, 0.02)
#     elif classname.find('BatchNorm') != -1:
#         m.weight.data.normal_(1.0, 0.02)
#         m.bias.data.fill_(0)

        
        
# lowlight_images_path="/kaggle/input/zero-dce-pytorchdataset/DICM/"
# lr=0.0001
# weight_decay=0.0001
# grad_clip_norm=0.1
# num_epochs=200
# train_batch_size=8
# val_batch_size=4
# num_workers=2
# snapshot_iter=10
# snapshots_folder="/kaggle/working/snapshots/"
# load_pretrain=False
# pretrain_dir="snapshots/Epoch99.pth"

# if not os.path.exists(snapshots_folder):
#     os.mkdir(snapshots_folder)




# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# DCE_net = enhance_net_nopool()
# DCE_net.to(device)
# DCE_net.apply(weights_init)

# if load_pretrain == True:
#     DCE_net.load_state_dict(torch.load(pretrain_dir))

# train_dataset = lowlight_loader(lowlight_images_path)

# train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=train_batch_size, shuffle=True, num_workers=num_workers, pin_memory=True)


# L_color = L_color()
# L_spa = L_spa()
# L_exp = L_exp(16,0.6)
# L_TV = L_TV()


# optimizer = torch.optim.Adam(DCE_net.parameters(), lr=lr, weight_decay=weight_decay)

# DCE_net.train()

# for epoch in range(num_epochs):
#     loop=tqdm(train_loader)
#     for iteration, img_lowlight in enumerate(loop):

#         img_lowlight = img_lowlight.to(device)
#         enhanced_image_1,enhanced_image,A  = DCE_net(img_lowlight)
        
#         Loss_TV = 200*(L_TV(A))

#         loss_spa = torch.mean(L_spa(enhanced_image, img_lowlight))

#         loss_col = 5*torch.mean(L_color(enhanced_image))

#         loss_exp = 10*torch.mean(L_exp(enhanced_image))


#         # best_loss
#         loss =  Loss_TV + loss_spa + loss_col + loss_exp
#         #


#         optimizer.zero_grad()
#         loss.backward()
#         torch.nn.utils.clip_grad_norm_(DCE_net.parameters(),grad_clip_norm)
#         optimizer.step()
#         loop.set_description(f'Epoch [{epoch+1}/{num_epochs}]')
#         loop.set_postfix(loss = loss.item())
#     if ((epoch+1) % snapshot_iter) == 0:
#          torch.save(DCE_net.state_dict(), snapshots_folder + "Epoch" + str(epoch) + '.pth') 



 
# # parser = argparse.ArgumentParser()

# # # Input Parameters
# # parser.add_argument('--lowlight_images_path', type=str, default="/kaggle/input/zero-dce-pytorchdataset/DICM/")
# # parser.add_argument('--lr', type=float, default=0.0001)
# # parser.add_argument('--weight_decay', type=float, default=0.0001)
# # parser.add_argument('--grad_clip_norm', type=float, default=0.1)
# # parser.add_argument('--num_epochs', type=int, default=200)
# # parser.add_argument('--train_batch_size', type=int, default=8)
# # parser.add_argument('--val_batch_size', type=int, default=4)
# # parser.add_argument('--num_workers', type=int, default=4)
# # parser.add_argument('--display_iter', type=int, default=10)
# # parser.add_argument('--snapshot_iter', type=int, default=10)
# # parser.add_argument('--snapshots_folder', type=str, default="snapshots/")
# # parser.add_argument('--load_pretrain', type=bool, default= False)
# # parser.add_argument('--pretrain_dir', type=str, default= "snapshots/Epoch99.pth")

# # config = parser.parse_args()

# # if not os.path.exists(config.snapshots_folder):
# #     os.mkdir(config.snapshots_folder)


# # train(config)








	


In [8]:
# torch.save(DCE_net.state_dict(), "./self_model.pth")

In [10]:
# import torch
# import torch.nn as nn
# import torchvision
# import torch.backends.cudnn as cudnn
# import torch.optim
# import os
# import sys
# import argparse
# import time
# import numpy as np
# from torchvision import transforms
# from PIL import Image
# import glob
# import time


 
# def comlowlight(image_path):
#     device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#     data_lowlight = Image.open(image_path)



#     data_lowlight = (np.asarray(data_lowlight)/255.0)


#     data_lowlight = torch.from_numpy(data_lowlight).float()
#     data_lowlight = data_lowlight.permute(2,0,1)
#     data_lowlight = data_lowlight.to(device).unsqueeze(0)

#     DCE_net = enhance_net_nopool().to(device)
#     DCE_net.load_state_dict(torch.load('./self_model.pth'))
#     start = time.time()
#     _,enhanced_image,_ = DCE_net(data_lowlight)

#     end_time = (time.time() - start)
#     print(end_time)
#     result_path = "/kaggle/working/selfmodel/"
#     if not os.path.exists(result_path):
#         os.mkdir(result_path)
#     print(result_path+image_path.split("/")[-1])
#     torchvision.utils.save_image(enhanced_image, result_path+image_path.split("/")[-1])

    
# with torch.no_grad():
#     filePath = '/kaggle/input/zero-dce-pytorchdataset/'

#     file_list = os.listdir(filePath)

#     for file_name in file_list:
#         test_list = glob.glob(filePath+file_name+"/*") 
#         for image in test_list:
#             # image = image
#             print(image)
#             comlowlight(image)





# 修改测试图片地址就行，测试各种图片

In [5]:
import torch
import torch.nn as nn
import torchvision
import torch.backends.cudnn as cudnn
import torch.optim
import os
import sys
import argparse
import time
import numpy as np
from torchvision import transforms
from PIL import Image
import glob
import time


 
def comlowlight(image_path):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    data_lowlight = Image.open(image_path)



    data_lowlight = (np.asarray(data_lowlight)/255.0)


    data_lowlight = torch.from_numpy(data_lowlight).float()
    data_lowlight = data_lowlight.permute(2,0,1)
    data_lowlight = data_lowlight.to(device).unsqueeze(0)

    DCE_net = enhance_net_nopool().to(device)
    DCE_net.load_state_dict(torch.load('./snapshots/newEpoch200.pth'))
    #start = time.time()
    _,enhanced_image,r = DCE_net(data_lowlight)
    print(r.shape)
    #end_time = (time.time() - start)
    #print(end_time)
    result_path = "./working/"
    if not os.path.exists(result_path):
        os.mkdir(result_path)
    print(result_path+image_path.split("/")[-1])
    #torchvision.utils.save_image(enhanced_image, result_path+image_path.split("/")[-1])

    
with torch.no_grad():
    filePath = '/home/tx2/Zero-DCE/test_data/1/'
#     file_list = os.listdir(filePath)

#     for file_name in file_list:
#         test_list = glob.glob(filePath+file_name+"/*") 
#         for image in test_list:
#             # image = image
#             print(image)
#             comlowlight(image)




    test_list = glob.glob(filePath+"/*") 
    for image in test_list:
        # image = image
#         print(image)
        t0 = time.time()
        comlowlight(image)
        t = time.time() - t0
        #print("finish")
        print("Prediction cost {:.4f}s".format(t))

torch.Size([1, 3, 640, 480])
./working/01.jpg
Prediction cost 10.8285s


/home/tx2/.local/lib/python3.6/site-packages/torch/nn/functional.py:1794: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


# 各种滤波方法

In [19]:
files = os.listdir("/home/tx2/Zero-DCE/test_data/New_vedio/")
for i in range(1,len(files)):
    #读取图片
    img = cv2.imread('/home/tx2/Zero-DCE/test_data/New_vedio/NewVedio%d.jpg'%i)     
    #img=cv2.blur(img,(5,5))#均值滤波
    #img=cv2.boxFilter(img,-1,(5,5),normalize=1)#方框滤波
    #img=cv2.GaussianBlur(img,(3,3),0)#高斯滤波
    img=cv2.medianBlur(img,3)#中值滤波
    cv2.imwrite('/home/tx2/Zero-DCE/test_data/fix_pic/NewVedio%d.jpg'%i, img)

# 释放资源

# tensorrt加速

In [8]:
import numpy as np
import tensorrt as trt
import pycuda.driver as cuda
import pycuda.autoinit
import time
from PIL import Image
def predict(batch): # result gets copied into output
    # transfer input data to device
    cuda.memcpy_htod_async(d_input, batch, stream)
    # execute model
    context.execute_async_v2(bindings, stream.handle, None)  # 此处采用异步推理。如果想要同步推理，需将execute_async_v2替换成execute_v2
    # transfer predictions back
    cuda.memcpy_dtoh_async(output, d_output, stream)
    # syncronize threads
    stream.synchronize()

    return output
# 1. 确定batch size大小，与导出的trt模型保持一致
BATCH_SIZE = 1        

# 2. 选择是否采用FP16精度，与导出的trt模型保持一致
# USE_FP16 = True                                         
# target_dtype = np.float16 if USE_FP16 else np.float32   
f = open("new_DCE_net_onnx_model.trt", "rb")    
   # 创建一个Runtime(传入记录器Logger)
runtime = trt.Runtime(trt.Logger())
engine = runtime.deserialize_cuda_engine(f.read())      # 从文件中加载trt引擎
context = engine.create_execution_context()             # 创建context


# 4. 分配input和output内存
input_batch = np.random.randn(BATCH_SIZE, 720, 1280, 3).astype(np.float32)
output = np.empty([BATCH_SIZE, 3,720,1280], dtype = np.float32)

d_input = cuda.mem_alloc(1 * input_batch.nbytes)
d_output = cuda.mem_alloc(1 * output.nbytes)

bindings = [int(d_input), int(d_output)]

stream = cuda.Stream()

data_lowlight = Image.open("/home/tx2/Zero-DCE/test_data/New_vedio/NewVedio214.jpg")
data_lowlight = (np.asarray(data_lowlight)/255.0)
data_lowlight = np.transpose(data_lowlight,(2,0,1))
data_lowlight=np.ascontiguousarray(data_lowlight,dtype=np.float32)
t0 = time.time()
pred=predict(data_lowlight)
t = time.time() - t0
print("Prediction cost {:.4f}s".format(t))

Prediction cost 0.4885s


In [9]:
filePath = '/home/tx2/Zero-DCE/test_data/actual_vedio/'
test_list = glob.glob(filePath+"/*") 
for image in test_list:
    data_lowlight = Image.open(image)
    data_lowlight = (np.asarray(data_lowlight)/255.0)
    data_lowlight = np.transpose(data_lowlight,(2,0,1))
    data_lowlight=np.ascontiguousarray(data_lowlight,dtype=np.float32)
    t0 = time.time()
    pred=predict(data_lowlight)
    t = time.time() - t0
    print("Prediction cost {:.4f}s".format(t))

Prediction cost 0.4035s
Prediction cost 0.4445s
Prediction cost 0.3742s
Prediction cost 0.3580s
Prediction cost 0.3609s
Prediction cost 0.3718s
Prediction cost 0.3772s
Prediction cost 0.3749s
Prediction cost 0.3657s
Prediction cost 0.3719s
Prediction cost 0.3761s
Prediction cost 0.3718s
Prediction cost 0.3616s
Prediction cost 0.3567s
Prediction cost 0.3618s
Prediction cost 0.3562s
Prediction cost 0.3578s
Prediction cost 0.3573s
Prediction cost 0.3592s
Prediction cost 0.3705s
Prediction cost 0.3694s
Prediction cost 0.3736s
Prediction cost 0.3735s
Prediction cost 0.3733s
Prediction cost 0.3707s
Prediction cost 0.3631s
Prediction cost 0.3588s
Prediction cost 0.3610s
Prediction cost 0.3607s
Prediction cost 0.3598s
Prediction cost 0.3576s
Prediction cost 0.3591s
Prediction cost 0.3713s
Prediction cost 0.3686s
Prediction cost 0.3733s
Prediction cost 0.3738s
Prediction cost 0.3770s
Prediction cost 0.3729s
Prediction cost 0.3758s
Prediction cost 0.3584s
Prediction cost 0.3588s
Prediction cost 

# tensorrt加速 480x640

In [6]:
import numpy as np
import tensorrt as trt
import pycuda.driver as cuda
import pycuda.autoinit
import time
from PIL import Image
def predict(batch): # result gets copied into output
    # transfer input data to device
    cuda.memcpy_htod_async(d_input, batch, stream)
    # execute model
    context.execute_async_v2(bindings, stream.handle, None)  # 此处采用异步推理。如果想要同步推理，需将execute_async_v2替换成execute_v2
    # transfer predictions back
    cuda.memcpy_dtoh_async(output, d_output, stream)
    # syncronize threads
    stream.synchronize()

    return output
# 1. 确定batch size大小，与导出的trt模型保持一致
BATCH_SIZE = 1        

# 2. 选择是否采用FP16精度，与导出的trt模型保持一致
# USE_FP16 = True                                         
# target_dtype = np.float16 if USE_FP16 else np.float32   
f = open("DCE_net_onnx_model_480x640.trt", "rb")    
   # 创建一个Runtime(传入记录器Logger)
runtime = trt.Runtime(trt.Logger())
engine = runtime.deserialize_cuda_engine(f.read())      # 从文件中加载trt引擎
context = engine.create_execution_context()             # 创建context


# 4. 分配input和output内存
input_batch = np.random.randn(BATCH_SIZE, 480, 640, 3).astype(np.float32)
output = np.empty([BATCH_SIZE, 3,480,640], dtype = np.float32)

d_input = cuda.mem_alloc(1 * input_batch.nbytes)
d_output = cuda.mem_alloc(1 * output.nbytes)

bindings = [int(d_input), int(d_output)]

stream = cuda.Stream()

data_lowlight = Image.open("/home/tx2/Zero-DCE/test_data/DICM/01.jpg")
data_lowlight = (np.asarray(data_lowlight)/255.0)
data_lowlight = np.transpose(data_lowlight,(2,0,1))
data_lowlight=np.ascontiguousarray(data_lowlight,dtype=np.float32)
t0 = time.time()
pred=predict(data_lowlight)
t = time.time() - t0
print("Prediction cost {:.4f}s".format(t))

Prediction cost 0.1467s


In [8]:
for i in range(1,10):
    t0 = time.time()
    pred=predict(data_lowlight)
    t = time.time() - t0
    print(t)

0.16081500053405762
0.11444997787475586
0.11325883865356445
0.12192463874816895
0.11071562767028809
0.12683391571044922
0.11155390739440918
0.12220597267150879
0.11282849311828613


# 保存onnx模型

In [6]:
import torch
import torchvision.models as models
import torch.onnx
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
DCE_net = enhance_net_nopool().to(device)
DCE_net.load_state_dict(torch.load('//home/tx2/Zero-DCE/snapshots/newEpoch200.pth'))
dummy_input=torch.randn(1,3, 480, 640).to(device)
torch.onnx.export(DCE_net, dummy_input, 
                  "DCE_net_onnx_model_480x640_r.onnx", verbose=False)

/home/tx2/.local/lib/python3.6/site-packages/torch/nn/functional.py:1794: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


# 时间下采样实时增强

In [4]:
import numpy as np
import tensorrt as trt
import pycuda.driver as cuda
import pycuda.autoinit
import time
from PIL import Image
import torch
import torchvision
def predict(batch): # result gets copied into output
    # transfer input data to device
    cuda.memcpy_htod_async(d_input, batch, stream)
    # execute model
    context.execute_async_v2(bindings, stream.handle, None)  # 此处采用异步推理。如果想要同步推理，需将execute_async_v2替换成execute_v2
    # transfer predictions back
    cuda.memcpy_dtoh_async(output, d_output, stream)
    # syncronize threads
    stream.synchronize()

    return output
# 1. 确定batch size大小，与导出的trt模型保持一致
BATCH_SIZE = 1        

# 2. 选择是否采用FP16精度，与导出的trt模型保持一致
# USE_FP16 = True                                         
# target_dtype = np.float16 if USE_FP16 else np.float32   
f = open("DCE_net_onnx_model_480x640_r.trt", "rb")    
   # 创建一个Runtime(传入记录器Logger)
runtime = trt.Runtime(trt.Logger())
engine = runtime.deserialize_cuda_engine(f.read())      # 从文件中加载trt引擎
context = engine.create_execution_context()             # 创建context


# 4. 分配input和output内存
input_batch = np.random.randn(BATCH_SIZE, 480, 640, 3).astype(np.float32)
output = np.empty( [BATCH_SIZE,24,640,480], dtype = np.float32)

d_input = cuda.mem_alloc(1 * input_batch.nbytes)
d_output = cuda.mem_alloc(1 * output.nbytes)

bindings = [int(d_input), int(d_output)]

stream = cuda.Stream()

data_lowlight = Image.open("/home/tx2/Zero-DCE/test_data/DICM/01.jpg")
data_lowlight = (np.asarray(data_lowlight)/255.0)
data_lowlight = np.transpose(data_lowlight,(2,0,1))
data_lowlight=np.ascontiguousarray(data_lowlight,dtype=np.float32)
r=predict(data_lowlight)
print(r.shape)
# r=torch.from_numpy(r)
# r1,r2,r3,r4,r5,r6,r7,r8 = torch.split(r, 3, dim=1)

# x = Image.open("/home/tx2/Zero-DCE/test_data/DICM/01.jpg")
# x = (np.asarray(x)/255.0)
# x = torch.from_numpy(x).float()
# x = x.permute(2,0,1)
# x = x.unsqueeze(0)

# x = x + r1*(torch.pow(x,2)-x)
# x = x + r2*(torch.pow(x,2)-x)
# x = x + r3*(torch.pow(x,2)-x)
# enhance_image_1 = x + r4*(torch.pow(x,2)-x)		
# x = enhance_image_1 + r5*(torch.pow(enhance_image_1,2)-enhance_image_1)		
# x = x + r6*(torch.pow(x,2)-x)	
# x = x + r7*(torch.pow(x,2)-x)
# enhance_image = x + r8*(torch.pow(x,2)-x)
# torchvision.utils.save_image(enhance_image, './working/tensorrtr8_1.jpg')
# print(enhance_image)

(1, 24, 640, 480)


In [60]:
import torch
import torchvision
import numpy as np
import tensorrt as trt
import pycuda.driver as cuda
import pycuda.autoinit
import time
from PIL import Image
import cv2

gst_str = ('nvarguscamerasrc ! '
            'video/x-raw(memory:NVMM), '
            'width=(int)480, height=(int)640, '
            'format=(string)NV12, framerate=(fraction)20/1 ! '
            'nvvidconv flip-method=0 ! '
            'video/x-raw, width=(int){}, height=(int){}, '
            'format=(string)BGRx ! '
            'videoconvert ! appsink').format(480, 640)  #width, height

cap = cv2.VideoCapture(gst_str)
if not cap.isOpened():
   print('Failed to open camera!')
flag = cap.isOpened()
i = 0

def predict(batch): # result gets copied into output
    # transfer input data to device
    cuda.memcpy_htod_async(d_input, batch, stream)
    # execute model
    context.execute_async_v2(bindings, stream.handle, None)  # 此处采用异步推理。如果想要同步推理，需将execute_async_v2替换成execute_v2
    # transfer predictions back
    cuda.memcpy_dtoh_async(output, d_output, stream)
    # syncronize threads
    stream.synchronize()

    return output



# 1. 确定batch size大小，与导出的trt模型保持一致
BATCH_SIZE = 1        

# 2. 选择是否采用FP16精度，与导出的trt模型保持一致
# USE_FP16 = True                                         
# target_dtype = np.float16 if USE_FP16 else np.float32   
f = open("DCE_net_onnx_model_480x640_r.trt", "rb")    
   # 创建一个Runtime(传入记录器Logger)
runtime = trt.Runtime(trt.Logger())
engine = runtime.deserialize_cuda_engine(f.read())      # 从文件中加载trt引擎
context = engine.create_execution_context()             # 创建context


# 4. 分配input和output内存
input_batch = np.random.randn(BATCH_SIZE, 480, 640, 3).astype(np.float32)
output = np.empty([BATCH_SIZE, 24,640,480], dtype = np.float32)

d_input = cuda.mem_alloc(1 * input_batch.nbytes)
d_output = cuda.mem_alloc(1 * output.nbytes)

bindings = [int(d_input), int(d_output)]
stream = cuda.Stream()

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

j=0

while (flag):
    ret, frame = cap.read()
    cv2.imshow("Capture", frame)
    cv2.imwrite("./test_data/actual_vedio_tensorrt/actual_vedio_tensorrt%d.jpg"%i, frame)
    if(i>=60 and i%20==0):
#         if(j==0):
#             data_lowlight = Image.open("./test_data/actual_vedio_tensorrt/actual_vedio_tensorrt%d.jpg"%(j*5))
#             data_lowlight = (np.asarray(data_lowlight)/255.0)
#             data_lowlight = np.transpose(data_lowlight,(2,0,1))
#             data_lowlight=np.ascontiguousarray(data_lowlight,dtype=np.float32)
#             a0=predict(data_lowlight)
#         else:
#             a0=a5

        data_lowlight = Image.open("./test_data/actual_vedio_tensorrt/actual_vedio_tensorrt%d.jpg"%(j*20))
        data_lowlight = (np.asarray(data_lowlight)/255.0)
        data_lowlight = np.transpose(data_lowlight,(2,0,1))
        data_lowlight=np.ascontiguousarray(data_lowlight,dtype=np.float32)
        a0=predict(data_lowlight)
        r=torch.from_numpy(a0)

        # j=j+1
        # data_lowlight = Image.open("./test_data/actual_vedio_tensorrt/actual_vedio_tensorrt%d.jpg"%(j*10-1))
        # data_lowlight = (np.asarray(data_lowlight)/255.0)
        # data_lowlight = np.transpose(data_lowlight,(2,0,1))
        # data_lowlight=np.ascontiguousarray(data_lowlight,dtype=np.float32)
        # a10=predict(data_lowlight)

        # ra10=torch.from_numpy(a10)
        r1,r2,r3,r4,r5,r6,r7,r8 = torch.split(r.to(device), 3, dim=1)
        for z in range(20):
        #     ra0.to(device)
        #     ra10.to(device)

            #r=(9-z)/9*ra0+(z)/9*ra10

            x = Image.open("./test_data/actual_vedio_tensorrt/actual_vedio_tensorrt%d.jpg"%(j*20+z))
            x = (np.asarray(x)/255.0)
            x = torch.from_numpy(x).float()
            x = x.permute(2,0,1)
            x = x.to(device).unsqueeze(0)

            x = x + r1*(torch.pow(x,2)-x)
            x = x + r2*(torch.pow(x,2)-x)
            x = x + r3*(torch.pow(x,2)-x)
            enhance_image_1 = x + r4*(torch.pow(x,2)-x)		

            x = enhance_image_1 + r5*(torch.pow(enhance_image_1,2)-enhance_image_1)		
            x = x + r6*(torch.pow(x,2)-x)	
            x = x + r7*(torch.pow(x,2)-x)
            enhance_image = x + r8*(torch.pow(x,2)-x)
            torchvision.utils.save_image(enhance_image, './final/%d.jpg'%(j*5+z-5))
        j=j+1

    i=i+1
    if cv2.waitKey(33) & 0xFF == 27 :
        break 
cap.release() # 释放摄像头
cv2.destroyAllWindows()# 释放并销毁窗口  




files = os.listdir("/home/tx2/Zero-DCE/test_data/actual_vedio_tensorrt/")
# 要被合成的多张图片所在文件夹
# 路径分隔符最好使用“/”,而不是“\”,“\”本身有转义的意思；或者“\\”也可以。
# 因为是文件夹，所以最后还要有一个“/”
# VideoWriter是cv2库提供的视频保存方法，将合成的视频保存到该路径中
# 'MJPG'意思是支持jpg格式图片
# fps = 5代表视频的帧频为5，如果图片不多，帧频最好设置的小一点
# (1280,720)是生成的视频像素1280*720，一般要与所使用的图片像素大小一致，否则生成的视频无法播放
# 定义保存视频目录名称和压缩格式，像素为1280*720
video = cv2.VideoWriter('/home/tx2/Zero-DCE/test_vedio/actual_test_tensorrt.mp4',cv2.VideoWriter_fourcc(*'mp4v'),30,(1280,720))

for i in range(1,len(files)):
    #读取图片
    img = cv2.imread('./working/actual_vedio_tensorrt%d.jpg'%i)     
   	# resize方法是cv2库提供的更改像素大小的方法
    # 将图片转换为1280*720像素大小
    #img = cv2.resize(img,(1280,720))
    # 写入视频
    video.write(img)

# # 释放资源
# video.release()
print("finish!")

finish!


In [13]:
import torch
import torchvision
import numpy as np
import tensorrt as trt
import pycuda.driver as cuda
import pycuda.autoinit
import time
from PIL import Image
import cv2

gst_str = ('nvarguscamerasrc ! '
            'video/x-raw(memory:NVMM), '
            'width=(int)480, height=(int)640, '
            'format=(string)NV12, framerate=(fraction)20/1 ! '
            'nvvidconv flip-method=0 ! '
            'video/x-raw, width=(int){}, height=(int){}, '
            'format=(string)BGRx ! '
            'videoconvert ! appsink').format(480, 640)  #width, height

cap = cv2.VideoCapture(gst_str)
if not cap.isOpened():
   print('Failed to open camera!')
flag = cap.isOpened()
i = 0

def predict(batch): # result gets copied into output
    # transfer input data to device
    cuda.memcpy_htod_async(d_input, batch, stream)
    # execute model
    context.execute_async_v2(bindings, stream.handle, None)  # 此处采用异步推理。如果想要同步推理，需将execute_async_v2替换成execute_v2
    # transfer predictions back
    cuda.memcpy_dtoh_async(output, d_output, stream)
    # syncronize threads
    stream.synchronize()

    return output



# 1. 确定batch size大小，与导出的trt模型保持一致
BATCH_SIZE = 1        

# 2. 选择是否采用FP16精度，与导出的trt模型保持一致
# USE_FP16 = True                                         
# target_dtype = np.float16 if USE_FP16 else np.float32   
f = open("DCE_net_onnx_model_480x640_r.trt", "rb")    
   # 创建一个Runtime(传入记录器Logger)
runtime = trt.Runtime(trt.Logger())
engine = runtime.deserialize_cuda_engine(f.read())      # 从文件中加载trt引擎
context = engine.create_execution_context()             # 创建context


# 4. 分配input和output内存
input_batch = np.random.randn(BATCH_SIZE, 480, 640, 3).astype(np.float32)
output = np.empty([BATCH_SIZE, 24,640,480], dtype = np.float32)

d_input = cuda.mem_alloc(1 * input_batch.nbytes)
d_output = cuda.mem_alloc(1 * output.nbytes)

bindings = [int(d_input), int(d_output)]
stream = cuda.Stream()

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

j=0

while (flag):
    ret, frame = cap.read()
    #cv2.imshow("Capture", frame)
    cv2.imwrite("./test_data/actual_vedio_tensorrt/actual_vedio_tensorrt%d.jpg"%i, frame)
    if(i%20==0):
#         if(j==0):
#             data_lowlight = Image.open("./test_data/actual_vedio_tensorrt/actual_vedio_tensorrt%d.jpg"%(j*5))
#             data_lowlight = (np.asarray(data_lowlight)/255.0)
#             data_lowlight = np.transpose(data_lowlight,(2,0,1))
#             data_lowlight=np.ascontiguousarray(data_lowlight,dtype=np.float32)
#             a0=predict(data_lowlight)
#         else:
#             a0=a5

        data_lowlight = Image.open("./test_data/actual_vedio_tensorrt/actual_vedio_tensorrt%d.jpg"%(j*20))
        data_lowlight = (np.asarray(data_lowlight)/255.0)
        data_lowlight = np.transpose(data_lowlight,(2,0,1))
        data_lowlight=np.ascontiguousarray(data_lowlight,dtype=np.float32)
        a0=predict(data_lowlight)
        r=torch.from_numpy(a0)
        r1,r2,r3,r4,r5,r6,r7,r8 = torch.split(r.to(device), 3, dim=1)
        j=j+1
        #     ra0.to(device)
        #     ra10.to(device)

            #r=(9-z)/9*ra0+(z)/9*ra10

    x = Image.open("./test_data/actual_vedio_tensorrt/actual_vedio_tensorrt%d.jpg"%i)
    x = (np.asarray(x)/255.0)
    x = torch.from_numpy(x).float()
    x = x.permute(2,0,1)
    x = x.to(device).unsqueeze(0)

    x = x + r1*(torch.pow(x,2)-x)
    x = x + r2*(torch.pow(x,2)-x)
    x = x + r3*(torch.pow(x,2)-x)
    enhance_image_1 = x + r4*(torch.pow(x,2)-x)		

    x = enhance_image_1 + r5*(torch.pow(enhance_image_1,2)-enhance_image_1)		
    x = x + r6*(torch.pow(x,2)-x)	
    x = x + r7*(torch.pow(x,2)-x)
    enhance_image = x + r8*(torch.pow(x,2)-x)
    torchvision.utils.save_image(enhance_image, './final/%d.jpg'%i)
    src=cv2.imread('./final/%d.jpg'%i)
    cv2.imshow('input_image', src)
    i=i+1
    
    if cv2.waitKey(33) & 0xFF == 27 :
        break 
cap.release() # 释放摄像头
cv2.destroyAllWindows()# 释放并销毁窗口  




# files = os.listdir("/home/tx2/Zero-DCE/test_data/actual_vedio_tensorrt/")
# # 要被合成的多张图片所在文件夹
# # 路径分隔符最好使用“/”,而不是“\”,“\”本身有转义的意思；或者“\\”也可以。
# # 因为是文件夹，所以最后还要有一个“/”
# # VideoWriter是cv2库提供的视频保存方法，将合成的视频保存到该路径中
# # 'MJPG'意思是支持jpg格式图片
# # fps = 5代表视频的帧频为5，如果图片不多，帧频最好设置的小一点
# # (1280,720)是生成的视频像素1280*720，一般要与所使用的图片像素大小一致，否则生成的视频无法播放
# # 定义保存视频目录名称和压缩格式，像素为1280*720
# video = cv2.VideoWriter('/home/tx2/Zero-DCE/test_vedio/actual_test_tensorrt.mp4',cv2.VideoWriter_fourcc(*'mp4v'),30,(1280,720))

# for i in range(1,len(files)):
#     #读取图片
#     img = cv2.imread('./working/actual_vedio_tensorrt%d.jpg'%i)     
#    	# resize方法是cv2库提供的更改像素大小的方法
#     # 将图片转换为1280*720像素大小
#     #img = cv2.resize(img,(1280,720))
#     # 写入视频
#     video.write(img)

# # 释放资源
# video.release()
print("finish!")

finish!


In [17]:
print(r1)

tensor([[[[-0.2069, -0.2491, -0.2688,  ..., -0.2849, -0.2850, -0.2852],
          [-0.2853, -0.2854, -0.2854,  ..., -0.2901, -0.2900, -0.2899],
          [-0.2898, -0.2897, -0.2897,  ..., -0.2928, -0.2927, -0.2925],
          ...,
          [-0.2448, -0.2396, -0.2358,  ..., -0.2500, -0.2461, -0.2419],
          [-0.2378, -0.2337, -0.2298,  ..., -0.2645, -0.2593, -0.2531],
          [-0.2469, -0.2411, -0.2353,  ..., -0.2592, -0.2421, -0.2035]],

         [[-0.2285, -0.2645, -0.2846,  ..., -0.3042, -0.3043, -0.3044],
          [-0.3045, -0.3046, -0.3046,  ..., -0.3102, -0.3101, -0.3100],
          [-0.3100, -0.3099, -0.3098,  ..., -0.3124, -0.3123, -0.3121],
          ...,
          [-0.2765, -0.2720, -0.2688,  ..., -0.2798, -0.2764, -0.2729],
          [-0.2697, -0.2664, -0.2635,  ..., -0.2906, -0.2856, -0.2798],
          [-0.2742, -0.2692, -0.2644,  ..., -0.2809, -0.2630, -0.2291]],

         [[-0.2792, -0.3055, -0.3253,  ..., -0.3489, -0.3490, -0.3491],
          [-0.3491, -0.3492, -

# 0-5帧0，5算其余用权重

In [12]:

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
j=0
data_lowlight = Image.open("./test_data/actual_vedio_tensorrt/actual_vedio_tensorrt%d.jpg"%(j*20))
data_lowlight = (np.asarray(data_lowlight)/255.0)
data_lowlight = np.transpose(data_lowlight,(2,0,1))
data_lowlight=np.ascontiguousarray(data_lowlight,dtype=np.float32)
a0=predict(data_lowlight)
r=torch.from_numpy(a0)

# j=j+1
# data_lowlight = Image.open("./test_data/actual_vedio_tensorrt/actual_vedio_tensorrt%d.jpg"%(j*10-1))
# data_lowlight = (np.asarray(data_lowlight)/255.0)
# data_lowlight = np.transpose(data_lowlight,(2,0,1))
# data_lowlight=np.ascontiguousarray(data_lowlight,dtype=np.float32)
# a10=predict(data_lowlight)

# ra10=torch.from_numpy(a10)
r1,r2,r3,r4,r5,r6,r7,r8 = torch.split(r.to(device), 3, dim=1)
for z in range(20):
#     ra0.to(device)
#     ra10.to(device)

    #r=(9-z)/9*ra0+(z)/9*ra10
    
    
    
    x = Image.open("./test_data/actual_vedio_tensorrt/actual_vedio_tensorrt%d.jpg"%(j*20+z))
    x = (np.asarray(x)/255.0)
    x = torch.from_numpy(x).float()
    x = x.permute(2,0,1)
    x = x.to(device).unsqueeze(0)
    
    x = x + r1*(torch.pow(x,2)-x)
    x = x + r2*(torch.pow(x,2)-x)
    x = x + r3*(torch.pow(x,2)-x)
    enhance_image_1 = x + r4*(torch.pow(x,2)-x)		
    
    x = enhance_image_1 + r5*(torch.pow(enhance_image_1,2)-enhance_image_1)		
    x = x + r6*(torch.pow(x,2)-x)	
    x = x + r7*(torch.pow(x,2)-x)
    enhance_image = x + r8*(torch.pow(x,2)-x)
    torchvision.utils.save_image(enhance_image, './final/%d.jpg'%(j*20+z))
    
    src=cv2.imread('./final/%d.jpg'%(j*20+z))       
    cv2.imshow('input_image', src)
    cv2.waitKey(100)
    cv2.destroyAllWindows()



